<a href="https://colab.research.google.com/github/vishal-burman/PyTorch-Architectures/blob/master/modeling_ResNet/test_sample_ResNet34.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/vishal-burman/PyTorch-Architectures.git

Cloning into 'PyTorch-Architectures'...
remote: Enumerating objects: 235, done.
remote: Counting objects: 100% (235/235), done.
remote: Compressing objects: 100% (151/151), done.
remote: Total 1173 (delta 115), reused 155 (delta 60), pack-reused 938
Receiving objects: 100% (1173/1173), 8.50 MiB | 1.90 MiB/s, done.
Resolving deltas: 100% (686/686), done.


In [2]:
%cd PyTorch-Architectures/modeling_ResNet/

/content/PyTorch-Architectures/modeling_ResNet


In [3]:
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from torchvision import datasets, transforms
from model import ResNet, BasicBlock

if torch.cuda.is_available():
  torch.cuda.cudnn.deterministic = True

In [4]:
# Hyperparameters

RANDOM_SEED = 1
LEARNING_RATE = 0.001
BATCH_SIZE = 128
NUM_EPOCHS = 10

NUM_FEATURES = 28 * 28
NUM_CLASSES = 10

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
grayscale = True

In [5]:
# MNIST Dataset

train_dataset = datasets.MNIST(
    root="data",
    train=True,
    transform=transforms.ToTensor(),
    download=True,
)
test_dataset = datasets.MNIST(
    root="data",
    train=False,
    transform=transforms.ToTensor(),
)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
)
test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
)

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw




Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [6]:
# Checking the dataset
for images, labels in train_loader:
  print("Image Batch Dimensions: ", images.shape)
  print("Label Batch Dimensions: ", labels.shape)
  break

Image Batch Dimensions:  torch.Size([128, 1, 28, 28])
Label Batch Dimensions:  torch.Size([128])


In [11]:
def resnet34(NUM_CLASSES):
  model = ResNet(
      block=BasicBlock,
      layers=[3, 4, 6, 3],
      num_classes=NUM_CLASSES,
      grayscale=grayscale,
  )
  return model

In [15]:
torch.manual_seed(RANDOM_SEED)
model = resnet34(NUM_CLASSES)
model.to(device)

params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Trainable Parameters: ", params)

Trainable Parameters:  21283530


In [16]:
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [ ]:
def compute_accuracy(model, data_loader, device):
  pass

start_time = time.time()
for epoch in range(NUM_EPOCHS):
  model.train()
  for idx, (features, targets) in enumerate(train_loader):
    features = features.to(device)
    targets = targets.to(device)

    logits, probas = model(features)
    loss = F.cross_entropy(logits, targets)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # LOGGING
    if idx % 50 == 0:
      print("Batch: %04d/%04d | Epoch: %04d/%04d | Cost: %.4f" % (idx, len(train_loader), epoch+1, NUM_EPOCHS, loss.item()))
  
  model.eval()
  with torch.set_grad_enabled(False):
    pass
